### Importaciones

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import janitor 

/home/gresuto/mambaforge/envs/curso-np-pd/lib/python3.11/site-packages/janitor/utils.py:84: FutureWarning: PandasArray has been renamed NumpyExtensionArray. Use that instead. This alias will be removed in a future version.
  @_expand_grid.register(pd.arrays.PandasArray)


### Carga de datos y tratamientos de valores faltantes o inconscistencias

#### Carga de datos

In [3]:
data_raw = pd.read_csv('euro2024_players.csv')
data_raw

,Name,Position,Age,Club,Height,Foot,Caps,Goals,MarketValue,Country
0,Marc-André ter Stegen,Goalkeeper,32,FC Barcelona,187,right,40,0,28000000,Germany
1,Manuel Neuer,Goalkeeper,38,Bayern Munich,193,right,119,0,4000000,Germany
2,Oliver Baumann,Goalkeeper,34,TSG 1899 Hoffenheim,187,right,0,0,3000000,Germany
3,Nico Schlotterbeck,Centre-Back,24,Borussia Dortmund,191,left,12,0,40000000,Germany
4,Jonathan Tah,Centre-Back,28,Bayer 04 Leverkusen,195,right,25,0,30000000,Germany
...,...,...,...,...,...,...,...,...,...,...
618,Adam Hlozek,Second Striker,21,Bayer 04 Leverkusen,188,right,31,2,12000000,Czech Republic
619,Patrik Schick,Centre-Forward,28,Bayer 04 Leverkusen,191,left,37,18,22000000,Czech Republic
620,Mojmír Chytil,Centre-Forward,25,SK Slavia Prague,187,-,12,4,6500000,Czech Republic
621,Jan Kuchta,Centre-Forward,27,AC Sparta Prague,185,right,20,3,5000000,Czech Republic


#### Limpiando datos

In [4]:
(
    data_raw.isnull()
    .pipe(
        lambda df: df[df==True].count()
    )
)

Name           0
Position       0
Age            0
Club           0
Height         0
Foot           3
Caps           0
Goals          0
MarketValue    0
Country        0
dtype: int64

In [5]:

data_raw['Foot'] = data_raw['Foot'].fillna('right')
data_raw['Foot'] = data_raw['Foot'].str.replace("-", "right", case=False, regex=False)

data = (
    data_raw.rename_columns(
    {
        'Name' : 'Nombre',
        'Position' : 'Posicion',
        'Age' : 'Edad',
        'Club' : 'Club',
        'Height' : 'Estatura(cm)',
        'Foot' : 'Pierna_Dom',
        'Caps' : 'Partidos',
        'Goals' : 'Goles',
        'MarketValue' : 'Valor_de_Mercado',
        'Country' : 'Pais'    
    }
    
)

    
)
data

,Nombre,Posicion,Edad,Club,Estatura(cm),Pierna_Dom,Partidos,Goles,Valor_de_Mercado,Pais
0,Marc-André ter Stegen,Goalkeeper,32,FC Barcelona,187,right,40,0,28000000,Germany
1,Manuel Neuer,Goalkeeper,38,Bayern Munich,193,right,119,0,4000000,Germany
2,Oliver Baumann,Goalkeeper,34,TSG 1899 Hoffenheim,187,right,0,0,3000000,Germany
3,Nico Schlotterbeck,Centre-Back,24,Borussia Dortmund,191,left,12,0,40000000,Germany
4,Jonathan Tah,Centre-Back,28,Bayer 04 Leverkusen,195,right,25,0,30000000,Germany
...,...,...,...,...,...,...,...,...,...,...
618,Adam Hlozek,Second Striker,21,Bayer 04 Leverkusen,188,right,31,2,12000000,Czech Republic
619,Patrik Schick,Centre-Forward,28,Bayer 04 Leverkusen,191,left,37,18,22000000,Czech Republic
620,Mojmír Chytil,Centre-Forward,25,SK Slavia Prague,187,right,12,4,6500000,Czech Republic
621,Jan Kuchta,Centre-Forward,27,AC Sparta Prague,185,right,20,3,5000000,Czech Republic


#### Traduciendo posiciones al espanol

In [6]:
data['Posicion'] = data['Posicion'].replace(
    {
        
        'Centre-Back' : 'Defensa-Central',
        'Central Midfield' :  'Centrocampista-Central',
        'Centre-Forward': 'Delantero-Centro',
        'Goalkeeper' : 'Portero',
        'Defensive Midfield' : 'Centrocampista-Defensivo',
        'Attacking Midfield' : 'Centrocampista-Ofensivo',
        'Left Winger' : 'Extremo-Izquierdo',
        'Right-Back' : 'Lateral-Derecho',
        'Right Winger' :  'Extremo-Derecho',
        'Left-Back' : 'Lateral-Izquierdo',
        'Right Midfield' : 'Interior-Derecho',
        'Second Striker': 'Segundo-Delantero',
        'Left Midfield': 'Interior-Izquierdo'
    }
)


### Preguntas a priori

#### ¿Como está distribuido la participación por posición?

In [7]:
data['Posicion'].value_counts()

Posicion
Defensa-Central             125
Centrocampista-Central       81
Delantero-Centro             77
Portero                      72
Centrocampista-Defensivo     50
Centrocampista-Ofensivo      48
Extremo-Izquierdo            44
Lateral-Derecho              43
Extremo-Derecho              35
Lateral-Izquierdo            33
Interior-Derecho              6
Segundo-Delantero             5
Interior-Izquierdo            4
Name: count, dtype: int64

#### ¿Cual es la distribucion de participacion de clubes?

In [8]:
data.Club.value_counts().head(25)

Club
Manchester City        13
Inter Milan            13
FC Barcelona           12
Real Madrid            12
Paris Saint-Germain    12
RB Leipzig             11
Bayern Munich          11
Bayer 04 Leverkusen    10
Arsenal FC             10
Liverpool FC            9
Juventus FC             9
SK Slavia Prague        9
AS Roma                 9
VfL Wolfsburg           8
AC Milan                8
Fenerbahce              8
Manchester United       8
Bologna FC 1909         8
Atlético de Madrid      7
Feyenoord Rotterdam     7
Brentford FC            7
Shakhtar Donetsk        7
Torino FC               6
RSC Anderlecht          6
Real Sociedad           6
Name: count, dtype: int64

#### Mayores goleadores internacionales de la competicion

In [9]:
numero_ranking = [1]
data.select_columns(['Nombre', 'Goles']).sort_values('Goles', ascending=False).set_index(data.index+1).head(30)

,Nombre,Goles
1,Cristiano Ronaldo,128
2,Romelu Lukaku,83
3,Robert Lewandowski,82
4,Harry Kane,63
5,Olivier Giroud,57
6,Aleksandar Mitrovic,57
7,Kylian Mbappé,47
8,Andriy Yarmolenko,46
9,Thomas Müller,45
10,Memphis Depay,45


#### Jugadores mas valiosos

In [10]:
data.select_columns(["Nombre", 'Valor_de_Mercado' ]).sort_values('Valor_de_Mercado', ascending=False).set_index(data.index+1).head(20)

,Nombre,Valor_de_Mercado
1,Jude Bellingham,180000000
2,Kylian Mbappé,180000000
3,Phil Foden,150000000
4,Bukayo Saka,140000000
5,Florian Wirtz,130000000
6,Rodri,120000000
7,Declan Rice,120000000
8,Jamal Musiala,120000000
9,Eduardo Camavinga,100000000
10,Harry Kane,100000000


### Preguntas de primer nivel

#### Construyendo Dataframe por equipos

In [11]:
data_equipos = (
    
    data.groupby(['Pais'])
    .sum()
    .drop(columns=['Nombre', 'Pierna_Dom', 'Club'])
    .assign(Total_Jugadores= data['Pais'].value_counts()) 
    .reset_index()
    .transform_column(
        column_name = 'Valor_de_Mercado',
        function = lambda x: round(x/1000000, 2)
    )
                )

data_equipos['Edad'] = data_equipos['Edad']/data_equipos['Total_Jugadores']

data_equipos['Estatura(cm)'] = data_equipos['Estatura(cm)']/data_equipos['Total_Jugadores']


data_equipos

,Pais,Posicion,Edad,Estatura(cm),Partidos,Goles,Valor_de_Mercado,Total_Jugadores
0,Albania,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,27.307692,183.615385,679,40,111.60,26
1,Austria,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,26.807692,183.192308,613,93,235.50,26
2,Belgium,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,26.880000,184.680000,949,177,584.50,25
3,Croatia,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,27.692308,184.115385,1152,147,327.70,26
4,Czech Republic,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,25.307692,185.538462,405,65,193.90,26
5,Denmark,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,27.692308,186.269231,1071,135,415.50,26
6,England,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,26.076923,182.461538,651,100,1515.00,26
7,France,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,26.880000,184.440000,836,192,1234.00,25
8,Georgia,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,27.153846,184.500000,750,64,160.15,26
9,Germany,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,28.115385,185.384615,906,134,851.00,26


#### Rankings

##### Ranking de estatura promedio de los equipos

In [12]:
altura_rank = data_equipos.loc[:,['Pais', 'Estatura(cm)']].sort_values('Estatura(cm)',ascending=False).set_index(data_equipos.index+1)
altura_rank

,Pais,Estatura(cm)
1,Serbia,186.769231
2,Denmark,186.269231
3,Czech Republic,185.538462
4,Germany,185.384615
5,Netherlands,185.192308
6,Slovenia,184.923077
7,Poland,184.814815
8,Belgium,184.680000
9,Georgia,184.500000
10,France,184.440000


#### Ranking de edad promedio de los equipos

In [13]:
edad_rank = data_equipos.loc[:,['Pais', 'Edad']].sort_values('Edad',ascending=False).set_index(data_equipos.index+1)
edad_rank

,Pais,Edad
1,Scotland,28.230769
2,Germany,28.115385
3,Croatia,27.692308
4,Switzerland,27.692308
5,Denmark,27.692308
6,Poland,27.555556
7,Hungary,27.423077
8,Serbia,27.384615
9,Slovenia,27.346154
10,Albania,27.307692


#### Ranking de experiencia por equipos

In [14]:
exp_rank = data_equipos.loc[:,['Pais', 'Partidos']].sort_values('Partidos',ascending=False).set_index(data_equipos.index+1)
exp_rank

,Pais,Partidos
1,Croatia,1152
2,Portugal,1113
3,Switzerland,1092
4,Denmark,1071
5,Belgium,949
6,Germany,906
7,Slovakia,862
8,Serbia,860
9,Ukraine,849
10,Netherlands,838


In [28]:
rel_exp = (
    data_equipos.loc[:,['Pais', 'Partidos','Edad' ]]
    .assign(Exp_Edad = data_equipos['Partidos']/data_equipos['Edad'])
    .sort_values('Partidos',ascending=False)
    .set_index(data_equipos.index+1)

)
rel_exp


,Pais,Partidos,Edad,Exp_Edad
1,Croatia,1152,27.692308,41.600000
2,Portugal,1113,26.961538,41.281027
3,Switzerland,1092,27.692308,39.433333
4,Denmark,1071,27.692308,38.675000
5,Belgium,949,26.880000,35.305060
6,Germany,906,28.115385,32.224350
7,Slovakia,862,27.269231,31.610719
8,Serbia,860,27.384615,31.404494
9,Ukraine,849,26.346154,32.224818
10,Netherlands,838,26.538462,31.576812


#### Ranking de goles por equipo

In [15]:
goles_rank = data_equipos.loc[:,['Pais', 'Goles']].sort_values('Goles',ascending=False).set_index(data_equipos.index+1)
goles_rank

,Pais,Goles
1,Portugal,225
2,France,192
3,Belgium,177
4,Poland,154
5,Croatia,147
6,Netherlands,141
7,Denmark,135
8,Germany,134
9,Serbia,129
10,Switzerland,125


#### Ranking de Valor de Mercado por equipos

In [58]:
VM_rank = data_equipos.loc[:,['Pais', 'Valor_de_Mercado']].sort_values('Valor_de_Mercado',ascending=False).assign(Pos=data_equipos.index+1)
VM_rank

,Pais,Valor_de_Mercado,Pos
6,England,1515.00,1
7,France,1234.00,2
14,Portugal,1054.50,3
20,Spain,965.50,4
9,Germany,851.00,5
12,Netherlands,845.00,6
11,Italy,705.50,7
2,Belgium,584.50,8
5,Denmark,415.50,9
23,Ukraine,379.00,10


### Preguntas de 2do Nivel

#### Agregando variables necesarias

##### Agregando Promedio de goles e Indice de Distribucion de goles

In [17]:
data_equipos = (
    data_equipos
    .assign(Prom_Goles= data_equipos['Goles']/data_equipos['Partidos'])
    .assign(Ind_Dist_Goles= data
            .select_columns(['Pais','Goles'])
            .groupby('Pais')
            .std()
    )
)

data_equipos

,Pais,Posicion,Edad,Estatura(cm),Partidos,Goles,Valor_de_Mercado,Total_Jugadores,Prom_Goles,Ind_Dist_Goles
0,Albania,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,27.307692,183.615385,679,40,111.60,26,0.058910,NaN
1,Austria,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,26.807692,183.192308,613,93,235.50,26,0.151713,NaN
2,Belgium,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,26.880000,184.680000,949,177,584.50,25,0.186512,NaN
3,Croatia,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,27.692308,184.115385,1152,147,327.70,26,0.127604,NaN
4,Czech Republic,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,25.307692,185.538462,405,65,193.90,26,0.160494,NaN
5,Denmark,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,27.692308,186.269231,1071,135,415.50,26,0.126050,NaN
6,England,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,26.076923,182.461538,651,100,1515.00,26,0.153610,NaN
7,France,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,26.880000,184.440000,836,192,1234.00,25,0.229665,NaN
8,Georgia,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,27.153846,184.500000,750,64,160.15,26,0.085333,NaN
9,Germany,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,28.115385,185.384615,906,134,851.00,26,0.147903,NaN


##### Agregando posiciones de los jugadores

In [18]:
posiciones_df = data.pivot_table(index= 'Pais', columns='Posicion', values='Nombre', aggfunc='count').reset_index()
posiciones_df = posiciones_df.fillna(0)


posiciones_resumen_df = (
    pd.DataFrame({
    'Pais' : posiciones_df['Pais']
    
    })
    .assign(
        Defensas= posiciones_df['Defensa-Central']
        + posiciones_df['Lateral-Derecho']
        + posiciones_df['Lateral-Izquierdo']
        
    ) 
    .assign(
        Centrocampista= posiciones_df['Centrocampista-Central'] 
        + posiciones_df['Centrocampista-Defensivo'] 
        + posiciones_df['Centrocampista-Ofensivo']
        + posiciones_df['Interior-Derecho']
        + posiciones_df['Interior-Izquierdo']
    )
    .assign(
        Atacantes= posiciones_df['Delantero-Centro'] 
        + posiciones_df['Extremo-Derecho'] 
        + posiciones_df['Extremo-Izquierdo']
        + posiciones_df['Segundo-Delantero']
    )
)



posiciones_df
posiciones_resumen_df.sort_values(by='Atacantes',ascending=False)
#posiciones_df['Centrocampista-Central'] + posiciones_df['Centrocampista-Defensivo'] + posiciones_df['Centrocampista-Ofensivo']
#data.select_columns(['Pais', 'Posicion']).groupby(['Pais']).value_counts()

,Pais,Defensas,Centrocampista,Atacantes
5,Denmark,9.0,6.0,8.0
7,France,8.0,6.0,8.0
8,Georgia,7.0,8.0,8.0
22,Turkiye,8.0,7.0,8.0
21,Switzerland,7.0,8.0,8.0
15,Romania,8.0,7.0,8.0
23,Ukraine,10.0,6.0,7.0
17,Serbia,8.0,8.0,7.0
2,Belgium,8.0,7.0,7.0
3,Croatia,8.0,8.0,7.0


#### Prediciendo formacion

In [19]:
%run scripts.py



#### Agregando valor de mercado plantilla titular

In [20]:
data_resumen_posicion = data.copy(deep=True)
data_resumen_posicion['Posicion'] = data_resumen_posicion['Posicion'].apply(posicion_generica)

equipo_titular = (
    data_resumen_posicion
    .transform_column(
        column_name = 'Valor_de_Mercado',
        function = lambda x: x/1000000
    )
    .rename_columns(
        {
            'Valor_de_Mercado' : 'Valor_de_Mercado(Millones)'
        }
    )
    .groupby(['Pais','Posicion','Nombre'])
    .sum()
    .sort_values(by=['Pais','Posicion','Valor_de_Mercado(Millones)'],ascending=False)
    #.reset_index()
    .drop(columns=['Club', 'Estatura(cm)', 'Pierna_Dom', 'Edad'])
    .groupby(['Pais', 'Posicion'], group_keys=False)
    .apply(filtrar_top_jugadores)
    .reset_index()
)



equipo_titular



,Pais,Posicion,Nombre,Partidos,Goles,Valor_de_Mercado(Millones)
0,Albania,Centrocampista,Kristjan Asllani,20,2,18.0
1,Albania,Centrocampista,Ernest Muci,10,3,13.0
2,Albania,Centrocampista,Nedim Bajrami,23,4,4.5
3,Albania,Defensa,Berat Djimsiti,57,1,10.0
4,Albania,Defensa,Marash Kumbulla,19,0,4.5
...,...,...,...,...,...,...
259,Ukraine,Defensa,Mykola Matvienko,64,0,18.0
260,Ukraine,Delantero,Artem Dovbyk,27,9,35.0
261,Ukraine,Delantero,Mykhaylo Mudryk,20,2,35.0
262,Ukraine,Delantero,Viktor Tsygankov,53,12,30.0


#### Rankings

##### Ranking posiciones

In [26]:
ranking_posiciones = posiciones_resumen_df.sort_values(by='Atacantes',ascending=False).set_index(data_equipos.index + 1)
ranking_posiciones

,Pais,Defensas,Centrocampista,Atacantes
1,Denmark,9.0,6.0,8.0
2,France,8.0,6.0,8.0
3,Georgia,7.0,8.0,8.0
4,Turkiye,8.0,7.0,8.0
5,Switzerland,7.0,8.0,8.0
6,Romania,8.0,7.0,8.0
7,Ukraine,10.0,6.0,7.0
8,Serbia,8.0,8.0,7.0
9,Belgium,8.0,7.0,7.0
10,Croatia,8.0,8.0,7.0


##### Ranking promedio de goles

In [59]:
GolProm_rank = data_equipos.select_columns(['Pais', 'Prom_Goles']).sort_values('Prom_Goles', ascending=False).assign(Pos=data_equipos.index+1)
GolProm_rank

,Pais,Prom_Goles,Pos
7,France,0.229665,1
14,Portugal,0.202156,2
13,Poland,0.186667,3
2,Belgium,0.186512,4
20,Spain,0.177570,5
12,Netherlands,0.168258,6
4,Czech Republic,0.160494,7
6,England,0.153610,8
1,Austria,0.151713,9
17,Serbia,0.150000,10


##### Ranking distribucion de goles

In [22]:
ranking_dist_goles = (
    data
    .select_columns(['Pais', 'Goles'])
    .rename_columns(
        {
            'Goles' : 'Ind_Dist_Goles'
        }
          
    )
    .groupby('Pais').std()
    .sort_values('Ind_Dist_Goles')
    .reset_index()
    .set_index(data_equipos.index+1)
)


#jugadoresXpaises = jugadoresXpaises.loc[:,['Country','Goals']]
#jugadoresXpaises.sort_values('Goals')
ranking_dist_goles

,Pais,Ind_Dist_Goles
1,Albania,1.881080
2,Italy,2.807956
3,Hungary,3.295685
4,Slovakia,3.568452
5,Scotland,3.572760
6,Romania,3.579643
7,Georgia,3.797165
8,Czech Republic,4.235564
9,Slovenia,4.308132
10,Turkiye,4.984437


##### Ranking valor de mercado equipo titular

In [23]:
ranking_equipo_titular_VM = (
    
    equipo_titular
    .select_columns(['Pais', 'Goles', 'Valor_de_Mercado(Millones)'])
    .groupby('Pais').sum()
    .sort_values('Valor_de_Mercado(Millones)',ascending=False)
    
)
ranking_equipo_titular_VM

,Goles,Valor_de_Mercado(Millones)
Pais,,
England,91,976.0
France,68,838.0
Portugal,56,655.0
Germany,41,621.0
Spain,26,615.0
Netherlands,25,535.0
Italy,36,460.0
Belgium,36,386.0
Denmark,39,302.0


##### Ranking de goles equipo titular

In [24]:
ranking_equipo_titular_G = (
    
    equipo_titular
    .select_columns(['Pais', 'Goles', 'Valor_de_Mercado(Millones)', 'Partidos'])
    .groupby('Pais').sum()
    .reset_index()
    #.sort_values('Prom_Goles',ascending=False)
    .set_index(data_equipos.index + 1)
    
)
hola = (
        ranking_equipo_titular_G
        .assign(Prom_Goles = ranking_equipo_titular_G['Goles']/ ranking_equipo_titular_G['Partidos'])
        .sort_values('Prom_Goles',ascending=False)
        .set_index(data_equipos.index + 1)
    )
hola

,Pais,Goles,Valor_de_Mercado(Millones),Partidos,Prom_Goles
1,Poland,117,146.5,424,0.275943
2,Serbia,87,238.5,320,0.271875
3,England,91,976.0,359,0.253482
4,France,68,838.0,318,0.213836
5,Czech Republic,37,131.0,219,0.168950
6,Austria,51,161.0,306,0.166667
7,Portugal,56,655.0,340,0.164706
8,Georgia,42,148.2,272,0.154412
9,Spain,26,615.0,172,0.151163
10,Denmark,39,302.0,281,0.138790


#### Conformando DataFrame con posiciones de ranking

In [60]:
ranking_general_df = (
    data_equipos
    .select_columns(['Pais'])
    
    
    
)
ranking_general_df['Pos_Rank_VM'] = VM_rank['Pos']
ranking_general_df['Pos_Rank_GP'] = GolProm_rank['Pos']
#ranking_general_df['Pos_Rank_PG'] = ranking_general_df.index.map(lambda x: GolProm_rank.index.get_loc(x) + 1 if x in GolProm_rank.index else None)

ranking_general_df

,Pais,Pos_Rank_VM,Pos_Rank_GP
0,Albania,23,24
1,Austria,15,9
2,Belgium,8,4
3,Croatia,11,13
4,Czech Republic,18,7
5,Denmark,9,14
6,England,1,8
7,France,2,1
8,Georgia,20,21
9,Germany,5,11
